In [ ]:
###  MODELO BUENO ###

import os
import torch
import argparse
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from torchvision import transforms, models
from torchvision.models import inception_v3
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from scipy.linalg import sqrtm

# CONFIGURACIÓN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
csv_path = "C:/Users/34629/TFG/single_label_dataset_without_H_no_O.csv"
weights_path = "C:/Users/34629/Downloads/RETFound_MAE/RETFound_cfp_weights.pth"
output_dir = "C:/Users/34629/Downloads/ddpm_retfound_outputs"
os.makedirs(output_dir, exist_ok=True)
image_size = 64
epochs = 100
timesteps = 100
batch_size = 8
minority_classes = ["G", "C", "A", "M"]

# DDPM BETA SCHEDULE
betas = torch.linspace(1e-4, 0.02, timesteps).to(device)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)

# MODELO
class RETFoundDDPM(nn.Module):
    def __init__(self):
        super().__init__()
        base_model = models.resnet18(weights=None)
        torch.serialization.add_safe_globals([argparse.Namespace])
        checkpoint = torch.load(weights_path, map_location=device, weights_only=False)
        base_model.load_state_dict(checkpoint["model"], strict=False)
        self.encoder = nn.Sequential(*list(base_model.children())[:-2])
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1)
        )
    def forward(self, x):
        x = self.encoder(x)
        return self.decoder(x)

# DATASET
class EyeDataset(Dataset):
    def __init__(self, df_subset):
        self.paths = df_subset['Image_Path'].tolist()
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert('RGB')
        return self.transform(img)

# FID + IS
inception = inception_v3(pretrained=True, transform_input=False).to(device)
inception.fc = nn.Identity()

def get_activations(images):
    images = F.interpolate(images, size=(299, 299))
    with torch.no_grad():
        preds = inception(images)
        if isinstance(preds, tuple):  # <-- Aquí detectamos la tupla
            preds = preds[0]
        return preds.cpu().numpy()


def calculate_fid(real, fake):
    mu1, sigma1 = real.mean(0), np.cov(real, rowvar=False)
    mu2, sigma2 = fake.mean(0), np.cov(fake, rowvar=False)
    covmean = sqrtm(sigma1 @ sigma2)
    return np.sum((mu1 - mu2) ** 2) + np.trace(sigma1 + sigma2 - 2 * covmean.real)

def calculate_is(fake_acts):
    p_yx = torch.softmax(torch.tensor(fake_acts), dim=1)
    p_y = p_yx.mean(0)
    kl = (p_yx * (p_yx.log() - p_y.log())).sum(1).mean()
    return torch.exp(kl).item()

# ENTRENAMIENTO POR CLASE
df = pd.read_csv(csv_path)

for cls in minority_classes:
    print(f"\n🔁 Entrenando clase {cls}...")
    cls_dir = os.path.join(output_dir, cls)
    os.makedirs(cls_dir, exist_ok=True)

    sub_df = df[df[cls] == 1].reset_index(drop=True)
    dataset = EyeDataset(sub_df)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = RETFoundDDPM().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

    best_fid = float("inf")
    fixed_noise = torch.randn(16, 3, image_size, image_size).to(device)

    for epoch in range(1, epochs + 1):
        model.train()
        for real in tqdm(loader, desc=f"Epoch {epoch}/{epochs}"):
            real = real.to(device)
            t = torch.randint(0, timesteps, (real.size(0),), device=device)
            noise = torch.randn_like(real)
            alpha_t = alphas_cumprod[t].view(-1, 1, 1, 1)
            noisy_img = torch.sqrt(alpha_t) * real + torch.sqrt(1 - alpha_t) * noise
            pred_noise = model(noisy_img)
            loss = F.mse_loss(pred_noise, noise)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            x = torch.randn_like(fixed_noise)
            for i in reversed(range(timesteps)):
                alpha = alphas[i]
                alpha_bar = alphas_cumprod[i]
                beta = betas[i]
                pred_noise = model(x)
                x = (1 / torch.sqrt(alpha)) * (x - ((1 - alpha) / torch.sqrt(1 - alpha_bar)) * pred_noise)
                if i > 0:
                    x += torch.sqrt(beta) * torch.randn_like(x)

            save_image(x, os.path.join(cls_dir, f"samples_epoch{epoch}.png"), nrow=4, normalize=True)

            real_batch = next(iter(loader))[:16].to(device)
            real_acts = get_activations(real_batch)
            fake_acts = get_activations(x[:16])
            fid = calculate_fid(real_acts, fake_acts)
            is_score = calculate_is(fake_acts)

            print(f"📊 Epoch {epoch}: FID={fid:.2f}, IS={is_score:.2f}")
            if fid < best_fid:
                best_fid = fid
                torch.save(model.state_dict(), os.path.join(cls_dir, f"DDPM_best_model.pt"))
                print("✅ Guardado nuevo mejor modelo.")

    print(f"🏁 Finalizado {cls} | Mejor FID: {best_fid:.2f}")


c:\Users\34629\TFG\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\34629\TFG\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



🔁 Entrenando clase G...


Epoch 1/100: 100%|██████████| 33/33 [00:20<00:00,  1.64it/s]


📊 Epoch 1: FID=617.15, IS=1.06
✅ Guardado nuevo mejor modelo.


Epoch 2/100: 100%|██████████| 33/33 [00:19<00:00,  1.73it/s]


...

Epoch 99/100: 100%|██████████| 30/30 [00:24<00:00,  1.20it/s]


📊 Epoch 99: FID=745.27, IS=1.13


Epoch 100/100: 100%|██████████| 30/30 [00:28<00:00,  1.06it/s]


📊 Epoch 100: FID=737.18, IS=1.13
🏁 Finalizado M | Mejor FID: 656.03


: 